# Herramientas de diagnostico

In [ ]:
import os
import sys
from kafka import KafkaConsumer, KafkaProducer
from kafka.errors import KafkaError

import redis

from mammut import apps
from mammut.apps import InterpreterMain_pb2
from mammut.apps import WorkingMemory_pb2
from mammut.apps import KAFKA_BROKER_STR

In [ ]:
mammut_api_url = "http://api.mammut.svc.cluster.local:21161"
redis_address = 'redis-redis-ha.redis.svc.cluster.local'
if 'true' in os.getenv('KTT_FRAMEWORK_IN_DOCKER', "").lower():
    mammut_api_url = "http://api:21161"
    redis_address = 'redis'
    apps.KAFKA_BROKER_STR = 'localhost:9092'
else 'mammut-framework' in current_dir:
    mammut_api_url = "http://localhost:21161"
    redis_address = 'localhost'
    apps.KAFKA_BROKER_STR = 'kafka:9092'

redis_port = 6379

## Kafka

In [ ]:
consumer = KafkaConsumer('i-thought', 'rt-issue', 's-issue', 'o-thought', 't-issue', 'rs-issue', 'o-compile', 'i-compile',
                         enable_auto_commit=True,
                         auto_offset_reset='latest',
                         group_id='diagnostic',
                         bootstrap_servers=[KAFKA_BROKER_STR],
#                          bootstrap_servers=['kafka:9092'],
                         api_version=(0, 10))

Para chequear que tareas de compilacion se han enviado, ejecutar el siguiente codigo.

In [ ]:
raw_messages = consumer.poll(timeout_ms=20000, max_records=5000)
for topic_partition, messages in raw_messages.items():
    for message in messages:
        print('Topic: ' + topic_partition.topic + 'Key: ' + str(message.key))

## Redis

In [ ]:
redis_client = r = redis.StrictRedis(host=redis_address, port=6379, db=0)

id_separator = '->'
mammut_id = 4328
room_id = 4710404336

pending_status_key = f'Issues{id_separator}{mammut_id}{id_separator}{room_id}'
halted_mammut_roomField_id = 'HaltedMachineRoom'
pending_thoughts = f'PendingThoughts{id_separator}{mammut_id}{id_separator}{room_id}'
verbs_mammut_id = f'Verbs{id_separator}{mammut_id}'
verb_vertex_id_field = f'{mammut_id}{id_separator}Llamar{id_separator}call{id_separator}2121234567{id_separator}None'
stack_scope_working_memory_mammut_room_id = f'StackScope{id_separator}{mammut_id}{id_separator}{room_id}'

In [ ]:
redis_client.info()

### Verbs

In [ ]:
redis_client.hget(verbs_mammut_id, verb_vertex_id_field)

### Halted room state

In [ ]:
redis_client.hget(pending_status_key, halted_mammut_roomField_id)

In [ ]:
redis_client.hgetall(pending_status_key)

### Pending thoughts

In [ ]:
peding_len = redis_client.llen(pending_thoughts)
redis_client.lrange(pending_thoughts, 0, peding_len)

In [ ]:
### Check conversation scope

In [ ]:
stack_scope_working_memory_len = redis_client.llen(stack_scope_working_memory_mammut_room_id)
stack_scope_working_memory_list = redis_client.lrange(stack_scope_working_memory_mammut_room_id, 0, stack_scope_working_memory_len)
stack_scope_working_memory_list

In [ ]:
working_memory_value = WorkingMemory_pb2.WorkingMemoryValue()
working_memory_value.ParseFromString(stack_scope_working_memory_list[0])
working_memory_value

In [ ]:
redis_client.hgetall(str(working_memory_value))
# redis_client.hgetall(stack_scope_working_memory_list[0])

In [ ]:
### Erase conversation scope

In [ ]:
redis_client.ltrim(stack_scope_working_memory_mammut_room_id, stack_scope_working_memory_len+1, stack_scope_working_memory_len)

### Delete halted room mark and pending thoughts

In [ ]:
redis_client.delete(pending_status_key)
redis_client.delete(pending_thoughts)

### Clear all working memory

In [ ]:
redis_client.flushall()